In [1]:
from __future__ import print_function, division
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import chromedriver_binary


# Flight Scraping

In [20]:
url = 'https://www.transtats.bts.gov/ONTIME/Departures.aspx'

response = requests.get(url)

assert response.status_code == 200

In [21]:
driver = webdriver.Chrome() 

In [22]:
# This function automates the process of downloading csvs from 'https://www.transtats.bts.gov/ONTIME/Departures.aspx'
# by changing airport/airline fields as needed.

def get_stats(driver, airport_name, airline_name, month_number):
    
    driver.get(url)
    driver.find_element_by_tag_name('body').send_keys(Keys.HOME) 
    time.sleep(5)
    #soup = BeautifulSoup(driver.page_source, 'html.parser') 
    send_button = driver.find_element_by_xpath('//*[@id="chkAllStatistics"]') 
    send_button.click() 
    driver.execute_script("window.scrollTo(0, 150)")
    
    #time.sleep(15)
    airport = driver.find_element_by_xpath('//*[@id="cboAirport"]')
    airport.click() 
    airport.send_keys(airport_name) 
    
    #time.sleep(15)
    airline = driver.find_element_by_xpath('//*[@id="cboAirline"]') 
    airline.click() 
    airline.send_keys(airline_name) 
    
   # time.sleep(15)
    m_pat = '//*[@id="chkMonths_' + str(month_number) + '"]'
    month = driver.find_element_by_xpath(m_pat) 
    month.click() 
    
   # time.sleep(15)
    days = driver.find_element_by_xpath('//*[@id="chkAllDays"]')
    days.click() 
    
    driver.execute_script("window.scrollTo(150, 500)")
   # time.sleep(15)
    years = driver.find_element_by_xpath('//*[@id="chkYears_31"]')
    years.click() 

    submit = driver.find_element_by_xpath('//*[@id="btnSubmit"]')
    submit.click() 
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(10)
    
    download = driver.find_element_by_xpath('//*[@id="DL_CSV"]') 
    download.click() 
    

In [ ]:
city = 'San Francisco'

#air_name = 'United'

plane_lines = ['Southwest', 'Delta', 'United', 'American']
#plane_lines1 = ['Delta', 'United', 'American']
#plane_lines2 = ['Southwest', 'Delta', 'American']

month_nums = [0,3,5,9]


for num in month_nums:
    for p_name in plane_lines:
        get_stats(driver, city, p_name, num) 
   

In [ ]:
for i in range(1,144): 
    
    tag = 'EMPTY'
    
    if i <= 15:
        tag = 'SFO'
    elif i > 15 and i <= 31:
        tag = 'SEA'
    elif i > 31 and i <= 47:
        tag = 'PHX'
    elif i > 47 and i <= 55:
        tag = 'JFK'
    elif i > 55 and i <= 63:
        tag = 'LGA'
    elif i > 63 and i<= 75:
        tag = 'IAH'
    elif i > 75 and i <= 87:
        tag = 'HNL'
    elif i > 87 and i <= 103:
        tag = 'BNA'
    elif i > 103 and i <= 115:
        tag = 'STL'
    elif i > 115 and i <= 127:
        tag = 'CMH'
    elif i > 127 and i <= 143:
        tag = 'CLE'
        
    url = 'Detailed_Statistics_Departures (' + str(i) + ')' + '.csv'
    print(url)  
    new_df = pd.read_csv(url, skiprows=6)
    new_df['Origin'] = tag
    df = df.append(new_df)

In [ ]:
df = df[df['Carrier Code'] != ' SOURCE: Bureau of Transportation Statistics'] 
#df.to_pickle("flight_stats")

# Weather Scraping

In [2]:
url = "https://www.wunderground.com/history/monthly/us/ca/san-francisco/KSFO/date/2018-1"

response = requests.get(url)

assert response.status_code == 200

In [3]:
driver = webdriver.Chrome() 

In [4]:
def easy_scrape(driver, my_cols, month_dict, my_url, tag):
    #my_url = all_urls[0][1]
    #current_tag = all_urls[0][0]
    driver.get(my_url)
    time.sleep(10)
    tables = pd.read_html(driver.page_source)
    
    first_df = pd.concat([tables[3], tables[4], tables[5], tables[6], tables[7], tables[8]], axis=1)
    first_df = first_df.iloc[1:]
    first_df.columns = my_cols
    first_df.reset_index(inplace=True, drop=True)
    month = tables[2][0][0]
    prefix = month_dict[month]
    
    date_list = []

    to_iter = {'Jan' : 32, 'Apr' : 31, 'Jun' : 31, 'Oct' : 32}
    
    for i in range (1,to_iter[month]):
        date = prefix + '/' + str(i) + '/' + '2018'
        date_list.append(date)

    first_df['Date'] = date_list
    first_df['Origin'] = tag
    
    return first_df

In [5]:
month_dict = {'Jan' : '01', 'Apr' : '04', 'Jun' : '06', 'Oct' : '10' }

In [6]:
airport_urls = ['https://www.wunderground.com/history/monthly/us/ca/san-francisco/KSFO/date/2018-',
                'https://www.wunderground.com/history/monthly/us/wa/seattle/KSEA/date/2018-',
                'https://www.wunderground.com/history/monthly/us/az/phoenix/KPHX/date/2018-',
                'https://www.wunderground.com/history/monthly/us/ny/new-york-city/KJFK/date/2018-',
                'https://www.wunderground.com/history/monthly/us/ny/new-york-city/KLGA/date/2018-',
                'https://www.wunderground.com/history/monthly/us/tx/houston/KIAH/date/2018-',
                'https://www.wunderground.com/history/monthly/us/hi/honolulu/PHNL/date/2018-',
                'https://www.wunderground.com/history/monthly/us/tn/nashville/KBNA/date/2018-',
                'https://www.wunderground.com/history/monthly/us/mo/st.-louis/KSTL/date/2018-',
                'https://www.wunderground.com/history/monthly/us/oh/columbus/KCMH/date/2018-',              
               ] 

In [7]:
main = pd.read_pickle('flight_stats')

In [8]:
port_names = main['Origin'].unique() .tolist()
port_names = port_names[:-1]
port_names

['SFO', 'SEA', 'PHX', 'JFK', 'LGA', 'IAH', 'HNL', 'BNA', 'STL', 'CMH']

In [9]:
suffix = ['1', '4', '6', '10']
all_urls = []  
index = -1
for port in airport_urls:
    index += 1
    for s in suffix:
        final_url = port + s
        all_urls.append((port_names[index], final_url))
    

In [10]:
all_urls[0:4]

[('SFO',
  'https://www.wunderground.com/history/monthly/us/ca/san-francisco/KSFO/date/2018-1'),
 ('SFO',
  'https://www.wunderground.com/history/monthly/us/ca/san-francisco/KSFO/date/2018-4'),
 ('SFO',
  'https://www.wunderground.com/history/monthly/us/ca/san-francisco/KSFO/date/2018-6'),
 ('SFO',
  'https://www.wunderground.com/history/monthly/us/ca/san-francisco/KSFO/date/2018-10')]

In [11]:
weather_df = pd.DataFrame()

In [ ]:
for link in all_urls:
    curr_df = easy_scrape(driver, my_cols, month_dict, link[1], link[0] )
    weather_df = weather_df.append(curr_df)
    

In [12]:
#weather_df.to_pickle('with_rain')

In [13]:
weather_df = pd.read_pickle('with_rain')

In [14]:
new_cols = ['Origin', 'Date', 'Min_Temperature', 'Avg_Temperature', 'Max_Temperature', 'Min_DewPoint',
       'Avg_DewPoint', 'Max_DewPoint', 'Min_Humidity', 'Avg_Humidity',
       'Max_Humidity', 'Min_Wind', 'Avg_Wind', 'Max_Wind', 'Min_Pressure',
       'Avg_Pressure', 'Max_Pressure', 'Precipitation']

In [15]:
weather_df = weather_df[new_cols]
weather_df.head()

,Origin,Date,Min_Temperature,Avg_Temperature,Max_Temperature,Min_DewPoint,Avg_DewPoint,Max_DewPoint,Min_Humidity,Avg_Humidity,Max_Humidity,Min_Wind,Avg_Wind,Max_Wind,Min_Pressure,Avg_Pressure,Max_Pressure,Precipitation
0,SFO,01/1/2018,57,52.1,45,50,46.8,43,93,82.3,67,5,1.7,0,30.2,30.2,30.1,0.00
1,SFO,01/2/2018,58,54.4,50,49,46.3,42,89,74.8,57,7,2.3,0,30.2,30.1,30.1,0.00
2,SFO,01/3/2018,55,53.4,52,51,48.5,45,93,83.3,74,8,2.5,0,30.1,30.0,30.0,0.00
3,SFO,01/4/2018,63,56.8,53,56,52.9,50,93,86.8,78,14,5.8,0,30.1,30.1,30.0,0.30
4,SFO,01/5/2018,63,56.7,53,59,53.2,50,93,88.2,83,16,5.5,0,30.2,30.2,30.1,0.03


In [16]:
def change_date(date):
    if len(date) == 9:
        date = date[:2] + '/0' + date[3:]
    return date

In [17]:
cp = weather_df.copy()
cp.reset_index(inplace=True,drop=True)
cp['Date'] = cp['Date'].apply(change_date)

In [18]:
cp.head()

,Origin,Date,Min_Temperature,Avg_Temperature,Max_Temperature,Min_DewPoint,Avg_DewPoint,Max_DewPoint,Min_Humidity,Avg_Humidity,Max_Humidity,Min_Wind,Avg_Wind,Max_Wind,Min_Pressure,Avg_Pressure,Max_Pressure,Precipitation
0,SFO,01/01/2018,57,52.1,45,50,46.8,43,93,82.3,67,5,1.7,0,30.2,30.2,30.1,0.00
1,SFO,01/02/2018,58,54.4,50,49,46.3,42,89,74.8,57,7,2.3,0,30.2,30.1,30.1,0.00
2,SFO,01/03/2018,55,53.4,52,51,48.5,45,93,83.3,74,8,2.5,0,30.1,30.0,30.0,0.00
3,SFO,01/04/2018,63,56.8,53,56,52.9,50,93,86.8,78,14,5.8,0,30.1,30.1,30.0,0.30
4,SFO,01/05/2018,63,56.7,53,59,53.2,50,93,88.2,83,16,5.5,0,30.2,30.2,30.1,0.03


In [23]:
#cp.to_pickle('with_rain_final')

# Combining Data

In [24]:
flight_df = pd.read_pickle('flight_stats')
weather_df = pd.read_pickle('with_rain_final')

In [25]:
flight_df.columns

Index(['Carrier Code', 'Date (MM/DD/YYYY)', 'Flight Number', 'Tail Number',
       'Destination Airport', 'Scheduled departure time',
       'Actual departure time', 'Scheduled elapsed time (Minutes)',
       'Actual elapsed time (Minutes)', 'Departure delay (Minutes)',
       'Wheels-off time', 'Taxi-Out time (Minutes)', 'Delay Carrier (Minutes)',
       'Delay Weather (Minutes)', 'Delay National Aviation System (Minutes)',
       'Delay Security (Minutes)', 'Delay Late Aircraft Arrival (Minutes)',
       'Origin'],
      dtype='object')

In [26]:
flight_df.head(5)

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Destination Airport,Scheduled departure time,Actual departure time,Scheduled elapsed time (Minutes),Actual elapsed time (Minutes),Departure delay (Minutes),Wheels-off time,Taxi-Out time (Minutes),Delay Carrier (Minutes),Delay Weather (Minutes),Delay National Aviation System (Minutes),Delay Security (Minutes),Delay Late Aircraft Arrival (Minutes),Origin
0,WN,01/01/2018,194.0,N8651A,MDW,05:15,05:15,245.0,236.0,0.0,05:28,13.0,0.0,0.0,0.0,0.0,0.0,SFO
1,WN,01/01/2018,246.0,N912WN,PHX,10:45,11:51,120.0,102.0,66.0,12:01,10.0,4.0,0.0,0.0,0.0,44.0,SFO
2,WN,01/01/2018,359.0,N8670A,DEN,18:30,18:33,145.0,145.0,3.0,18:50,17.0,0.0,0.0,0.0,0.0,0.0,SFO
3,WN,01/01/2018,367.0,N480WN,BUR,14:15,14:58,70.0,77.0,43.0,15:24,26.0,3.0,0.0,7.0,0.0,40.0,SFO
4,WN,01/01/2018,377.0,N8662F,MDW,15:10,15:29,245.0,250.0,19.0,15:45,16.0,19.0,0.0,5.0,0.0,0.0,SFO


In [27]:
weather_df.head(5)

,Origin,Date,Min_Temperature,Avg_Temperature,Max_Temperature,Min_DewPoint,Avg_DewPoint,Max_DewPoint,Min_Humidity,Avg_Humidity,Max_Humidity,Min_Wind,Avg_Wind,Max_Wind,Min_Pressure,Avg_Pressure,Max_Pressure,Precipitation
0,SFO,01/01/2018,57,52.1,45,50,46.8,43,93,82.3,67,5,1.7,0,30.2,30.2,30.1,0.00
1,SFO,01/02/2018,58,54.4,50,49,46.3,42,89,74.8,57,7,2.3,0,30.2,30.1,30.1,0.00
2,SFO,01/03/2018,55,53.4,52,51,48.5,45,93,83.3,74,8,2.5,0,30.1,30.0,30.0,0.00
3,SFO,01/04/2018,63,56.8,53,56,52.9,50,93,86.8,78,14,5.8,0,30.1,30.1,30.0,0.30
4,SFO,01/05/2018,63,56.7,53,59,53.2,50,93,88.2,83,16,5.5,0,30.2,30.2,30.1,0.03


In [28]:
flight_df = flight_df.rename(columns={"Date (MM/DD/YYYY)" : "Date"})


In [29]:
new_cols = ['Origin',
             'Date',
             'Carrier Code',
             'Flight Number',
             'Tail Number',
             'Destination Airport',
             'Scheduled departure time',
             'Actual departure time',
             'Scheduled elapsed time (Minutes)',
             'Actual elapsed time (Minutes)',
             'Departure delay (Minutes)',
             'Wheels-off time',
             'Taxi-Out time (Minutes)',
             'Delay Carrier (Minutes)',
             'Delay Weather (Minutes)',
             'Delay National Aviation System (Minutes)',
             'Delay Security (Minutes)',
             'Delay Late Aircraft Arrival (Minutes)',
             ]

In [30]:
flight_df = flight_df[new_cols]

In [31]:
edited = flight_df[flight_df['Origin'] != 'CLE']
mer = edited.merge(weather_df, on=['Origin', 'Date']) 


In [32]:
mer['Origin'].unique()

array(['SFO', 'SEA', 'PHX', 'JFK', 'LGA', 'IAH', 'HNL', 'BNA', 'STL',
       'CMH'], dtype=object)

In [33]:
def get_gates(place):
    number_gates = {'SFO':115, 'SEA':80, 'PHX':116, 'JFK':128, 'LGA':72, 'IAH':130, 'HNL':47, 'BNA':42, 'STL':86, 'CMH':37}
    return number_gates[place]
    

In [34]:
mer['Num Gates'] = mer['Origin'].apply(get_gates) 

In [35]:
mer.sample(10) 

,Origin,Date,Carrier Code,Flight Number,Tail Number,Destination Airport,Scheduled departure time,Actual departure time,Scheduled elapsed time (Minutes),Actual elapsed time (Minutes),...,Avg_Humidity,Max_Humidity,Min_Wind,Avg_Wind,Max_Wind,Min_Pressure,Avg_Pressure,Max_Pressure,Precipitation,Num Gates
117672,LGA,04/29/2018,WN,2352.0,N751SW,MKE,11:40,11:38,150.0,119.0,...,64.4,37,25,12.1,0,30.0,29.9,29.8,0.22,72
138154,IAH,06/26/2018,UA,1956.0,N69830,LAX,09:50,09:53,201.0,199.0,...,75.6,52,15,6.7,0,29.9,29.9,29.8,0.51,130
49129,SEA,10/02/2018,UA,459.0,N410UA,ORD,07:00,06:59,245.0,236.0,...,80.7,58,22,12.6,5,29.5,29.3,29.3,0.30,80
131325,IAH,04/23/2018,UA,2067.0,N485UA,TPA,14:35,14:26,127.0,127.0,...,63.6,38,17,6.3,0,30.0,29.9,29.8,0.00,130
43818,SEA,06/04/2018,DL,2860.0,N376DA,LAS,11:30,11:34,153.0,150.0,...,66.7,37,8,5.1,0,29.7,29.6,29.6,0.01,80
1110,SFO,01/04/2018,UA,2162.0,N817UA,EUG,20:13,20:04,95.0,80.0,...,86.8,78,14,5.8,0,30.1,30.1,30.0,0.30,115
145412,HNL,01/02/2018,UA,650.0,N37470,SFO,17:00,16:58,310.0,312.0,...,56.0,45,17,10.1,6,30.1,30.1,30.0,0.00,47
105060,JFK,04/21/2018,AA,1587.0,N928NN,PHX,16:30,16:36,346.0,326.0,...,48.9,22,20,11.7,0,30.4,30.4,30.3,0.00,128
94633,PHX,10/20/2018,AA,686.0,N508AY,SFO,17:15,17:10,121.0,106.0,...,37.3,27,17,11.1,5,28.8,28.7,28.7,0.00,116
92535,PHX,10/15/2018,WN,913.0,N729SW,SAN,11:50,11:51,70.0,73.0,...,43.8,23,16,9.6,0,28.8,28.8,28.7,0.00,116


In [36]:
#x = pd.Series(['Geeks', 'for', 'Geeks']) 

#unique_flights = pd.DataFrame(mer['Tail Number'].unique())

#def get_age(tail_num):
#    try:
#        url = 'https://registry.faa.gov/aircraftinquiry/NNum_Results.aspx?NNumbertxt=' + str(tail_num)
#        tables = pd.read_html(url, match = 'A/W Date')
#        cert_date = tables[0][1][2]
#        return cert_date
#    except:
#        return 'NaN'
    
#unique_flights['Cert_Date'] = unique_flights['Tail Number'].apply(get_age)    

In [38]:

mer = mer[mer['Tail Number'] != 'N389AA'] 

In [39]:
check = pd.read_pickle('flight_ages')

check = check[check['Cert_Date'] != 'NaN']

final = mer.merge(check, on=['Tail Number'])

In [40]:
final.columns



Index(['Origin', 'Date', 'Carrier Code', 'Flight Number', 'Tail Number',
       'Destination Airport', 'Scheduled departure time',
       'Actual departure time', 'Scheduled elapsed time (Minutes)',
       'Actual elapsed time (Minutes)', 'Departure delay (Minutes)',
       'Wheels-off time', 'Taxi-Out time (Minutes)', 'Delay Carrier (Minutes)',
       'Delay Weather (Minutes)', 'Delay National Aviation System (Minutes)',
       'Delay Security (Minutes)', 'Delay Late Aircraft Arrival (Minutes)',
       'Min_Temperature', 'Avg_Temperature', 'Max_Temperature', 'Min_DewPoint',
       'Avg_DewPoint', 'Max_DewPoint', 'Min_Humidity', 'Avg_Humidity',
       'Max_Humidity', 'Min_Wind', 'Avg_Wind', 'Max_Wind', 'Min_Pressure',
       'Avg_Pressure', 'Max_Pressure', 'Precipitation', 'Num Gates',
       'Cert_Date'],
      dtype='object')

In [41]:
dropped_cols = ['Scheduled elapsed time (Minutes)', 'Actual elapsed time (Minutes)', 'Flight Number']

final = final.drop(columns=dropped_cols)



In [42]:
final.head()

,Origin,Date,Carrier Code,Tail Number,Destination Airport,Scheduled departure time,Actual departure time,Departure delay (Minutes),Wheels-off time,Taxi-Out time (Minutes),...,Max_Humidity,Min_Wind,Avg_Wind,Max_Wind,Min_Pressure,Avg_Pressure,Max_Pressure,Precipitation,Num Gates,Cert_Date
0,SFO,01/01/2018,WN,N8651A,MDW,05:15,05:15,0.0,05:28,13.0,...,67,5,1.7,0,30.2,30.2,30.1,0.00,115,10/16/2014
1,SFO,04/08/2018,WN,N8651A,PDX,20:30,21:17,47.0,21:44,27.0,...,45,26,12.0,0,30.2,30.2,30.1,0.10,115,10/16/2014
2,SFO,06/01/2018,WN,N8651A,PDX,06:20,06:18,-2.0,06:49,31.0,...,39,26,11.6,0,30.1,30.0,30.0,0.00,115,10/16/2014
3,SFO,06/02/2018,WN,N8651A,PHX,11:25,11:23,-2.0,11:33,10.0,...,33,23,10.3,0,30.0,29.9,29.9,0.00,115,10/16/2014
4,SFO,06/18/2018,WN,N8651A,MDW,09:25,09:24,-1.0,09:39,15.0,...,55,24,11.4,0,30.1,30.0,30.0,0.00,115,10/16/2014


In [44]:
t_list = ['Origin', 'Date', 'Num Gates', 'Carrier Code', 'Tail Number', 'Cert_Date']
cols = final.columns.tolist()

new_cols = t_list + cols[4:-2]

final = final[new_cols] 

final = final[final['Cert_Date'] != 'None']

final.head()

,Origin,Date,Num Gates,Carrier Code,Tail Number,Cert_Date,Destination Airport,Scheduled departure time,Actual departure time,Departure delay (Minutes),...,Min_Humidity,Avg_Humidity,Max_Humidity,Min_Wind,Avg_Wind,Max_Wind,Min_Pressure,Avg_Pressure,Max_Pressure,Precipitation
0,SFO,01/01/2018,115,WN,N8651A,10/16/2014,MDW,05:15,05:15,0.0,...,93,82.3,67,5,1.7,0,30.2,30.2,30.1,0.00
1,SFO,04/08/2018,115,WN,N8651A,10/16/2014,PDX,20:30,21:17,47.0,...,83,64.6,45,26,12.0,0,30.2,30.2,30.1,0.10
2,SFO,06/01/2018,115,WN,N8651A,10/16/2014,PDX,06:20,06:18,-2.0,...,80,61.7,39,26,11.6,0,30.1,30.0,30.0,0.00
3,SFO,06/02/2018,115,WN,N8651A,10/16/2014,PHX,11:25,11:23,-2.0,...,69,58.3,33,23,10.3,0,30.0,29.9,29.9,0.00
4,SFO,06/18/2018,115,WN,N8651A,10/16/2014,MDW,09:25,09:24,-1.0,...,77,67.6,55,24,11.4,0,30.1,30.0,30.0,0.00


In [45]:
switch = final.copy()

switch = switch.rename(columns={"Min_Temperature" : "Max_Temp", "Max_Temperature" : "Min_Temp"})
switch = switch.rename(columns={"Min_DewPoint" : "Max_Dew", "Max_DewPoint" : "Min_Dew"})
switch = switch.rename(columns={"Min_Humidity" : "Max_Humid", "Max_Humidity" : "Min_Humid"})
switch = switch.rename(columns={"Min_Wind" : "Max_Wind", "Max_Wind" : "Min_Wind"})
switch = switch.rename(columns={"Min_Pressure" : "Max_Pressure", "Max_Pressure" : "Min_Pressure"})

switch.head()

,Origin,Date,Num Gates,Carrier Code,Tail Number,Cert_Date,Destination Airport,Scheduled departure time,Actual departure time,Departure delay (Minutes),...,Max_Humid,Avg_Humidity,Min_Humid,Max_Wind,Avg_Wind,Min_Wind,Max_Pressure,Avg_Pressure,Min_Pressure,Precipitation
0,SFO,01/01/2018,115,WN,N8651A,10/16/2014,MDW,05:15,05:15,0.0,...,93,82.3,67,5,1.7,0,30.2,30.2,30.1,0.00
1,SFO,04/08/2018,115,WN,N8651A,10/16/2014,PDX,20:30,21:17,47.0,...,83,64.6,45,26,12.0,0,30.2,30.2,30.1,0.10
2,SFO,06/01/2018,115,WN,N8651A,10/16/2014,PDX,06:20,06:18,-2.0,...,80,61.7,39,26,11.6,0,30.1,30.0,30.0,0.00
3,SFO,06/02/2018,115,WN,N8651A,10/16/2014,PHX,11:25,11:23,-2.0,...,69,58.3,33,23,10.3,0,30.0,29.9,29.9,0.00
4,SFO,06/18/2018,115,WN,N8651A,10/16/2014,MDW,09:25,09:24,-1.0,...,77,67.6,55,24,11.4,0,30.1,30.0,30.0,0.00


In [46]:
switch = switch.rename(columns={'Avg_Temperature' : 'Avg_Temp'})
#switch.to_pickle('FINAL3')